## Variationsalgorithmen21

Dieser Kurs behandelt die Besonderheiten von Variationsalgorithmen und kurzfristigen hybriden quantenklassischen Algorithmen basierend auf dem Variationssatz der Quantenmechanik. Diese Algorithmen können den Nutzen nutzen, den heutige nicht fehlertolerante Quantencomputer bieten, was sie zu idealen Kandidaten für die Erzielung [von Quantenvorteilen](gloss:advantage) macht.

In diesem Kurs werden wir Folgendes untersuchen:

- Jeder Schritt im Design-Workflow des Variationsalgorithmus
- Mit jedem Schritt verbundene Kompromisse
- So verwenden Sie [Qiskit Runtime-Primitive](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html) zur Optimierung von Geschwindigkeit und Genauigkeit

Während dieser Kurs als Ausgangspunkt für Forscher und Entwickler gedacht ist, um den Nutzen von Quantencomputern zu erkunden, können Sie sich gerne mit den theoretischen und grundlegenden Kenntnissen rund um Quantencomputer im Allgemeinen im [Abschnitt „Grundlagen der Quanteninformation und -berechnung“](https://qiskit.org/learn/course/basics-quantum-information/) (auch als [Reihe](https://www.youtube.com/playlist?list=PLOFEBzvs-VvqKKMXX4vbi4EB1uaErFMSO) verfügbar) vertraut machen [von YouTube-Videos](https://www.youtube.com/playlist?list=PLOFEBzvs-VvqKKMXX4vbi4EB1uaErFMSO) ).

## Vereinfachter Hybrid-Workflow

![Variationsfluss](images/variational_workflow.png)

Variationsalgorithmen umfassen mehrere modulare Komponenten, die basierend auf Algorithmen-, Software- und Hardware-Fortschritten kombiniert und optimiert werden können. Dazu gehören eine *Kostenfunktion* , die ein spezifisches Problem mit einem Satz von Parametern beschreibt, ein *Ansatz* , um den Suchraum mit diesen Parametern auszudrücken, und ein *Optimierer* , um den Suchraum iterativ zu erkunden. Während jeder Iteration wertet der Optimierer die Kostenfunktion mit den aktuellen Parametern aus und wählt die Parameter der nächsten Iteration aus, bis er zu einer optimalen Lösung [konvergiert](gloss:converge) . Der hybride Charakter dieser Familie von Algorithmen ergibt sich aus der Tatsache, dass die Kostenfunktionen mithilfe von Quantenressourcen bewertet und durch klassische Ressourcen optimiert werden.

1. **Initialisierungsproblem** : Variationsalgorithmen beginnen damit, den Quantencomputer in einem *Standardzustand* $|0\rangle$ zu initialisieren und ihn dann in einen gewünschten (nicht parametrisierten) Zustand $|\rho\rangle$ umzuwandeln, den wir *Referenzzustand* nennen werden.

    Diese Transformation wird durch die Anwendung eines einheitlichen Referenzoperators $U_R$ auf den Standardzustand dargestellt, so dass $U_R|0\rangle = |\rho\rangle$.

2. **Ansatz vorbereiten** : Um mit der iterativen Optimierung vom Standardzustand $|0\rangle$ zum Zielzustand $|\psi(\vec\theta)\rangle$ zu beginnen, müssen wir eine *Variationsform* $U_V(\vec\theta)$ definieren um eine Sammlung parametrisierter Zustände darzustellen, die unser Variationsalgorithmus untersuchen soll.

    Wir bezeichnen jede bestimmte Kombination aus Referenzzustand und Variationsform als Ansatz, so dass: $U_A(\vec\theta) := U_V(\vec\theta) U_R$. Die Ansätze werden letztendlich die Form parametrisierter Quantenschaltkreise annehmen, die in der Lage sind, den Standardzustand $|0\rangle$ in den Zielzustand $|\psi(\vec\theta)\rangle$ zu bringen.

    Alles in allem werden wir haben:

    $$
   \begin{aligned}
   |0\rangle \xrightarrow{U_R} U_R|0\rangle

   & = |\rho\rangle \xrightarrow{U_V(\vec{\theta})} U_A(\vec{\theta})|0\rangle \\[1mm]

   & = U_V(\vec{\theta})U_R|0\rangle \\[1mm]

   & = U_V(\vec{\theta})|\rho\rangle \\[1mm]

   & = |\psi(\vec{\theta})\rangle \\[1mm]

   \end{aligned}
   $$

3. **Kostenfunktion auswerten** : Wir können unser Problem in eine *Kostenfunktion* $C(\vec\theta)$ als lineare Kombination von Pauli-Operatoren kodieren, die auf einem Quantensystem ausgeführt wird. Dabei kann es sich zwar um Informationen über ein physikalisches System wie Energie oder Spin handeln, wir können aber auch nicht-physikalische Probleme kodieren. Wir können Qiskit Runtime-Grundelemente nutzen, um Rauschen durch Fehlerunterdrückung und -minderung zu bekämpfen und gleichzeitig unsere Kostenfunktion zu bewerten.

4. **Parameter optimieren** : Auswertungen werden an einen klassischen Computer weitergeleitet, wo ein klassischer Optimierer sie analysiert und den nächsten Wertesatz für die Variationsparameter auswählt. Wenn wir bereits eine optimale Lösung haben, können wir diese als *Ausgangspunkt* $\vec\theta_0$ festlegen, um unsere Optimierung *zu starten* . Die Verwendung dieses *Anfangszustands* $|\psi(\vec\theta_0)\rangle$ könnte unserem Optimierer helfen, schneller eine gültige Lösung zu finden.

5. **Passen Sie die Ansatzparameter mit den Ergebnissen an und führen Sie sie erneut aus** : Der gesamte Prozess wird wiederholt, bis die Finalisierungskriterien des klassischen Optimierers erfüllt sind und ein optimaler Satz von Parameterwerten $\vec\theta^ *$ zurückgegeben wird. Der vorgeschlagene Lösungszustand für unser Problem ist dann $|\psi(\vec\theta^* )\rangle = U_A(\vec\theta^*)|0\rangle$.

## Variationssatz

Ein gemeinsames Ziel von Variationsalgorithmen besteht darin, den Quantenzustand mit dem niedrigsten oder höchsten Eigenwert einer bestimmten Observablen zu finden. Eine wichtige Erkenntnis, die wir verwenden werden, ist der *Variationssatz* der Quantenmechanik. Bevor wir auf die vollständige Aussage eingehen, wollen wir uns mit der mathematischen Intuition befassen, die dahinter steckt.

### Mathematische Intuition für Energie und Grundzustände

In der Quantenmechanik liegt Energie in Form einer Quantenobservable vor, die üblicherweise als *Hamiltonoperator* bezeichnet wird und die wir mit $\hat{\mathcal{H}}$ bezeichnen. Betrachten wir seine [spektrale Zerlegung](gloss:decomposition) :

$$
\hat{\mathcal{H}} = \sum_{k=0}^{N-1} \lambda_k |\phi_k\rangle \langle \phi_k|
$$

wobei $N$ die Dimensionalität des Zustandsraums ist, $\lambda_{k}$ der $k$-te Eigenwert oder physikalisch das $k$-te Energieniveau ist und $|\phi_k\rangle$ ist der entsprechende [Eigenzustand](gloss:eigenstate) : $\hat{\mathcal{H}}|\phi_k\rangle = \lambda_k |\phi_k\rangle$, die erwartete Energie eines Systems im (normalisierten) Zustand $|\psi\rangle$ :

$$
\begin{aligned}
\langle \psi | \hat{\mathcal{H}} | \psi \rangle

& = \langle \psi |\bigg(\sum_{k=0}^{N-1} \lambda_k |\phi_k\rangle \langle \phi_k|\bigg) | \psi \rangle \\[1mm]

& = \sum_{k=0}^{N-1} \lambda_k \langle \psi |\phi_k\rangle \langle \phi_k| \psi \rangle \\[1mm]

& = \sum_{k=0}^{N-1} \lambda_k |\langle \psi |\phi_k\rangle|^2 \\[1mm]

\end{aligned}
$$

Wenn wir das $\lambda_0\leq \lambda_k, \forall k$ berücksichtigen, haben wir:

$$
\begin{aligned}
\langle \psi | \hat{\mathcal{H}} | \psi \rangle

& = \sum_{k=0}^{N-1} \lambda_k |\langle \psi |\phi_k\rangle|^2 \\[1mm]

& \geq  \sum_{k=0}^{N-1} \lambda_0 |\langle \psi |\phi_k\rangle|^2 \\[1mm]

& = \lambda_0 \sum_{k=0}^{N-1} |\langle \psi |\phi_k\rangle|^2 \\[1mm]

& = \lambda_0 \\[1mm]

\end{aligned}
$$

Da `{latex} \{ |\phi_k\rangle \}_{k=0}^{N-1}` eine Orthonormalbasis ist, beträgt die Wahrscheinlichkeit der Messung von $|\phi_{k} \rangle$ $p_k = |\ langle \psi |\phi_{k} \rangle |^2$, und die Summe aller Wahrscheinlichkeiten ist so, dass $\sum_{k=0}^{N-1} |\langle \psi |\phi_k\rangle| ^2 = \sum_{k=0}^{N-1}p_k = 1$. Kurz gesagt, die erwartete Energie jedes Systems ist höher als die niedrigste Energie oder Grundzustandsenergie:

$$
\langle \psi | \hat{\mathcal{H}} | \psi \rangle \geq \lambda_0.
$$

Das obige Argument gilt für jeden gültigen (normalisierten) Quantenzustand $|\psi\rangle$, daher ist es durchaus möglich, parametrisierte Zustände $|\psi(\vec\theta)\rangle$ zu berücksichtigen, die von einem Parametervektor $\ abhängen vec\theta$. Hier kommt der „variative“ Teil ins Spiel. Wenn wir eine Kostenfunktion betrachten, die durch $C(\vec\theta) := \langle \psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle$ gegeben ist und es minimieren möchten, wird das Minimum immer erfüllen:

$$
\min_{\vec\theta} C(\vec\theta) = 
\min_{\vec\theta} \langle \psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle \geq \lambda_0.
$$

Der Mindestwert von $C(\vec\theta)$ wird am nächsten an $\lambda_0$ herankommen, wenn man die parametrisierten Zustände $|\psi(\vec\theta)\rangle$ verwendet, und Gleichheit wird nur erreicht wenn es einen Parametervektor $\vec\theta^ *$ gibt, so dass $|\psi(\vec\theta^* )\rangle = |\phi_0\rangle$

### Variationssatz der Quantenmechanik

Wenn der (normalisierte) Zustand $|\psi\rangle$ eines Quantensystems von einem Parametervektor $\vec\theta$ abhängt, dann ist die optimale Näherung des Grundzustands (d. h. des Eigenzustands $|\phi_0\rangle$ mit dem minimaler Eigenwert $\lambda_0$) ist derjenige, der den [Erwartungswert](gloss:expectation) des Hamilton-Operators $\hat{\mathcal{H}}$ minimiert:

$$
\langle \hat{\mathcal{H}} \rangle(\vec\theta) := 
\langle \psi(\vec\theta) |\hat{\mathcal{H}}| \psi(\vec\theta) \rangle \geq 
\lambda_0
$$

Der Grund, warum der Variationssatz in Bezug auf Energieminima angegeben wird, liegt darin, dass er eine Reihe mathematischer Annahmen beinhaltet:

- Aus physikalischen Gründen muss eine endliche untere Schranke für die Energie $E \geq \lambda_0 &gt; -\infty$ existieren, auch für $N\rightarrow\infty$.
- Obergrenzen gibt es im Allgemeinen nicht.

Mathematisch gesehen gibt es jedoch über diese Annahmen hinaus nichts Besonderes am Hamilton-Operator $\hat{\mathcal{H}}$, sodass der Satz auf andere Quantenobservablen und deren Eigenzustände verallgemeinert werden kann, sofern sie denselben Einschränkungen folgen. Beachten Sie außerdem, dass, wenn endliche Obergrenzen existieren, dieselben mathematischen Argumente für die Maximierung von Eigenwerten angeführt werden könnten, indem Untergrenzen gegen Obergrenzen ausgetauscht werden.

In dieser Lektion haben Sie die allgemeine Sicht auf Variationsalgorithmen kennengelernt. In den folgenden Lektionen werden wir jeden Schritt und die damit verbundenen Kompromisse detaillierter untersuchen.